In [1]:


!pip install tensorflow-privacy==0.7.3



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 251 kB 16.7 MB/s 


In [73]:
#from __future__ import absolute_import, division, print_function

import sys
import logging
import math
import datetime
import random
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
#from tensorflow.keras.utils import to_categorical

#import tensorflow_privacy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow_privacy.privacy.analysis.rdp_accountant import (
    compute_rdp,
    get_privacy_spent,
)
from tensorflow_privacy.privacy.optimizers import dp_optimizer
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackType

In [140]:
noise_multiplier=0.01
epochs=200
learning_rate=0.1


In [141]:
df = pd.read_csv("student-mat.csv",sep=";")
#df_stud_por = pd.read_csv("student-por.csv",sep=";")

In [142]:
pd.set_option('display.max_columns', 52)
df.select_dtypes(include='object')

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,GP,F,U,GT3,A,at_home,teacher,course,mother,yes,no,no,no,yes,yes,no,no
1,GP,F,U,GT3,T,at_home,other,course,father,no,yes,no,no,no,yes,yes,no
2,GP,F,U,LE3,T,at_home,other,other,mother,yes,no,yes,no,yes,yes,yes,no
3,GP,F,U,GT3,T,health,services,home,mother,no,yes,yes,yes,yes,yes,yes,yes
4,GP,F,U,GT3,T,other,other,home,father,no,yes,yes,no,yes,yes,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,U,LE3,A,services,services,course,other,no,yes,yes,no,yes,yes,no,no
391,MS,M,U,LE3,T,services,services,course,mother,no,no,no,no,no,yes,yes,no
392,MS,M,R,GT3,T,other,other,course,other,no,no,no,no,no,yes,no,no
393,MS,M,R,LE3,T,services,other,course,mother,no,no,no,no,no,yes,yes,no


In [143]:
df_encode = pd.get_dummies(df)

In [144]:
y=df_encode.G3
exclude_var=['G3']
print(exclude_var)
x_data=df_encode.drop(columns=exclude_var)

['G3']


In [145]:
y.value_counts()

10    56
11    47
0     38
15    33
8     32
13    31
12    31
9     28
14    27
16    16
6     15
18    12
7      9
5      7
17     6
19     5
20     1
4      1
Name: G3, dtype: int64

In [146]:
ybin = np.asarray(y)

In [147]:
ybin[ybin<10] = 0
ybin[ybin>=10] = 1

In [148]:
from sklearn.model_selection import train_test_split


In [149]:
# min-max normalization
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values

In [150]:
x_train, x_test, y_train, y_test = train_test_split(x, ybin, test_size=0.2, random_state=0)

# get validation dataset
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.2, random_state=0
)

In [151]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((252, 58), (252,), (79, 58), (79,))

In [152]:
y_test

array([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0])

In [153]:
y_train

array([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [154]:
y_valid

array([0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1])

In [155]:
len(x.columns)

58

In [156]:
training_size = x_train.shape[0]
training_size

252

In [157]:
# from here https://github.com/VectorInstitute/PETs-Bootcamp/blob/main/DP_TensorFlowPrivacy/TFP_HeartDisease_KerasMLP_GridSearch.ipynb
def train(
    noise_multiplier,
    l2_norm_clip,
    batch_size,
    microbatches,
    x_train,
    y_train,
    dpsgd=True,
    learning_rate=learning_rate,
    epochs=epochs,
    model_dir=None,
    print_outputs=True,
    verbose=1,
):

    if dpsgd and batch_size % microbatches != 0:
        raise ValueError("Number of microbatches should divide evenly batch_size")

    # Define a sequential Keras model
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(40, input_dim=58, activation="relu"),
            tf.keras.layers.Dense(60, activation="relu"),
            tf.keras.layers.Dense(20, activation="relu"),
            #tf.keras.layers.Dense(1, activation="sigmoid"),
            #V2 for mia attack we need logit output
            tf.keras.layers.Dense(1),
        ]
    )

    if dpsgd:
        optimizer = DPKerasSGDOptimizer(
            l2_norm_clip=l2_norm_clip,
            noise_multiplier=noise_multiplier,
            num_microbatches=microbatches,
            learning_rate=learning_rate,
        )
        # Compute vector of per-example loss rather than its mean over a minibatch.
        #loss = tf.keras.losses.BinaryCrossentropy(reduction=tf.losses.Reduction.NONE)
        loss = tf.keras.losses.BinaryCrossentropy(reduction=tf.losses.Reduction.NONE,from_logits=True) #V2

    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
        #loss = tf.keras.losses.BinaryCrossentropy()
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=True) #V2

    # F.MIA'S ATTACK USE THESE PARAMETERS
    '''
    # specify parameters
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    '''

    # Compile model with Keras
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


    display(model.summary())

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, histogram_freq=1
    )

    # Train model with Keras
    history=model.fit(
        x_train,
        y_train,
        epochs=epochs,
        validation_data=(x_valid, y_valid),
        batch_size=batch_size,
        verbose=verbose,
        callbacks=[tensorboard_callback],
    )

    # plot accuracy for the first model
    plt.plot(history.history['accuracy'], label='acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.legend();
    plt.ylim(0,1)

    score_train = model.evaluate(x_train, y_train, verbose=verbose)
    score_valid = model.evaluate(x_valid, y_valid, verbose=verbose)
    score_test = model.evaluate(x_test, y_test, verbose=verbose)

    weights = model.get_weights()

    # Compute the privacy budget expended.
    # // is integer division
    if dpsgd:
        eps = compute_epsilon(
            epochs * training_size // batch_size,
            training_size=training_size,
            noise_multiplier=noise_multiplier,
            batch_size=batch_size,
        )

    else:
        eps = "non-private SGD"

    if print_outputs:
        print(
            "\nhyperparamters: learning rate = "
            + str(learning_rate)
            + ", noise_multiplier = "
            + str(noise_multiplier)
            + ", l2_norm_clip = "
            + str(l2_norm_clip)
            + ", epochs = "
            + str(epochs)
            + ", batch_size = "
            + str(batch_size)
            + ", microbatches = "
            + str(microbatches)
        )

        print("  training loss: %.2f" % score_train[0])
        print("  training accuracy: %.2f" % score_train[1])

        print("  validation loss: %.2f" % score_valid[0])
        print("  validation accuracy: %.2f" % score_valid[1])

        print("  test loss: %.2f" % score_test[0])
        print("  test accuracy: %.2f" % score_test[1])
        
        Y_pred = model.predict(x_test)
        Y_pred = np.where(Y_pred > 0.5, 1,0)#np.argmax(Y_pred,axis=1)
        print(classification_report(y_test, Y_pred))   

        if dpsgd:
            print("For delta=0.00413223, the current epsilon is: %.2f" % eps)
        else:
            print("Trained with vanilla non-private SGD optimizer")

    return ((score_train, score_valid, score_test, eps, weights, Y_pred),model)

In [158]:
def compute_epsilon(steps, training_size, noise_multiplier, batch_size):
    """
    Computes epsilon value for given hyperparameters.

    Parameters required:
      steps: Number of steps the optimizer takes over the training data
             steps = FLAGS.epochs * training_size// FLAGS.batch_size

      Noise multiplier:
          the amount of noise sampled and added to gradients during training
    """
    if noise_multiplier == 0.0:
        return float("inf")

    """ 
  Delta: for (epsilon, delta)-DP
    Delta bounds the probability of our privacy guarantee not holding.  
    rule of thumb for delta is to set it to less than the inverse of the training data size
    so I opted for it to equal to 1.1*training size
  """
    training_delta = 1 / (training_size * 1.1)

    """
  We need to define a list of orders, at which the Rényi divergence will be computed
  if you want epsilon between 1-10 and your delta is fixed
  your orders must cover the range between 1+ln(1/delta)/10 and 1+ln(1/delta)/1 
  """
    orders = np.linspace(
        1 + math.log(1.0 / training_delta) / 10,
        1 + math.log(1.0 / training_delta) / 1,
        num=100,
    )

    """ 
  Sampling ratio q:
    the probability of an individual training point being included in a minibatch
    sampling_probability = FLAGS.batch_size / training_size
  """
    sampling_probability = batch_size / training_size

    """ 
  compute Renyi Differential Privacy, a generalization of pure differential privacy
  RDP is well suited to analyze DP guarantees provided by sampling followed by Gaussian noise addition, 
  which is how gradients are randomized in the TFP implementation of the DP-SGD optimizer.
  """
    rdp = compute_rdp(
        q=sampling_probability,
        noise_multiplier=noise_multiplier,
        steps=steps,
        orders=orders,
    )

    return get_privacy_spent(orders, rdp, target_delta=training_delta)[0]

## Train model

In [ ]:
(training_outputs,model) = train(
    noise_multiplier=noise_multiplier,
    l2_norm_clip=1,
    batch_size=22,
    microbatches=11,
    x_train=x_train,
    y_train=y_train,
    dpsgd=False,
    learning_rate=0.01,
    epochs=200,
    model_dir=None,
    print_outputs=True,
)

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_160 (Dense)           (None, 40)                2360      
                                                                 
 dense_161 (Dense)           (None, 60)                2460      
                                                                 
 dense_162 (Dense)           (None, 20)                1220      
                                                                 
 dense_163 (Dense)           (None, 1)                 21        
                                                                 
Total params: 6,061
Trainable params: 6,061
Non-trainable params: 0
_________________________________________________________________


None

Epoch 1/200
12/12 [==============================] - 1s 21ms/step - loss: 0.6655 - accuracy: 0.3452 - val_loss: 0.6532 - val_accuracy: 0.2812
Epoch 2/200
12/12 [==============================] - 0s 8ms/step - loss: 0.6473 - accuracy: 0.4365 - val_loss: 0.6417 - val_accuracy: 0.3750
Epoch 3/200
12/12 [==============================] - 0s 10ms/step - loss: 0.6392 - accuracy: 0.4683 - val_loss: 0.6349 - val_accuracy: 0.4219
Epoch 4/200
12/12 [==============================] - 0s 8ms/step - loss: 0.6342 - accuracy: 0.5079 - val_loss: 0.6299 - val_accuracy: 0.5625
Epoch 5/200
12/12 [==============================] - 0s 8ms/step - loss: 0.6297 - accuracy: 0.5516 - val_loss: 0.6258 - val_accuracy: 0.5938
Epoch 6/200
12/12 [==============================] - 0s 8ms/step - loss: 0.6262 - accuracy: 0.6151 - val_loss: 0.6242 - val_accuracy: 0.6250
Epoch 7/200
12/12 [==============================] - 0s 8ms/step - loss: 0.6236 - accuracy: 0.6627 - val_loss: 0.6227 - val_accuracy: 0.6562
Epoch 8/200

In [ ]:
# dpsgd model
training_outputs = train(
    noise_multiplier=noise_multiplier,
    l2_norm_clip=1,
    batch_size=22,
    microbatches=1,
    x_train=x_train,
    y_train=y_train,
    dpsgd=True,
    learning_rate=0.01,
    epochs=epochs,
    #epochs=200,
    model_dir=None,
    print_outputs=True,
)

![image.png](attachment:28f51ce6-7447-473d-81b0-524b19486a42.png)
![image.png](attachment:d2ccc47d-6b01-4c9f-9356-b171ad155205.png)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
logits_train = model.predict(x_train)
print('Predict on test...')
logits_test = model.predict(x_test)

print('Apply softmax to get probabilities from logits...')
prob_train = tf.nn.sigmoid(np.array(logits_train).flatten())
prob_test = tf.nn.sigmoid(np.array(logits_test).flatten())

print('Compute losses...')
#cce = tf.keras.backend.categorical_crossentropy
bce = tf.keras.backend.binary_crossentropy
constant = tf.keras.backend.constant

#y_train_onehot = tf.keras.utils.to_categorical(y_train)
#y_test_onehot = tf.keras.utils.to_categorical(y_test)


In [ ]:
loss_train = bce(constant(y_train), constant(prob_train), from_logits=False).numpy()
loss_test = bce(constant(y_test), constant(prob_test), from_logits=False).numpy()

In [ ]:
y_train.shape,prob_train.shape

In [ ]:
# define what variables our attacker should have access to
attack_input = AttackInputData(
  logits_train = logits_train,
  logits_test = logits_test,
  loss_train = loss_train,
  loss_test = loss_test,
  labels_train = y_train,
  labels_test = y_test
)

In [ ]:
# how should the data be sliced
slicing_spec = SlicingSpec(
    entire_dataset = True,
    by_class = True,
    by_percentiles = False,
    by_classification_correctness = True)

In [ ]:
# define the type of attacker model that we want to use
attack_types = [
    AttackType.THRESHOLD_ATTACK,
    AttackType.LOGISTIC_REGRESSION
]

In [ ]:
# run the attack
attacks_result = mia.run_attacks(attack_input=attack_input,
                                 slicing_spec=slicing_spec,
                                 attack_types=attack_types)

In [ ]:
# summary by data slice (the best performing attacks per slice are presented)
print(attacks_result.summary(by_slices=True))

In [ ]:
# plot the curve, we see that the attacker is much better than random guessing
import tensorflow_privacy.privacy.membership_inference_attack.plotting as plotting
print(plotting.plot_roc_curve(attacks_result.get_result_with_max_auc().roc_curve))